# Hospital Site Matching Analysis

This notebook attempts to match anonymous hospital sites from the A&E project dataset with real hospital locations in Scotland.

## Approach:
1. Load the anonymous site data (Site_X, Site_Y coordinates)
2. Load real hospital data with geographical information
3. Convert hospitals to coordinate system used in the project
4. Perform geographical matching based on proximity
5. Validate matches using additional characteristics

## Data Sources:
- `OR_AE2_Project_Adjusted.xlsx`: Anonymous hospital sites with coordinates
- `hospitals.csv`: Real hospital database
- Shapefile data for coordinate conversion

## 1. Import Libraries and Setup

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Try to import geographical libraries
try:
    import geopandas as gpd
    from shapely.geometry import Point
    geo_available = True
    print("✓ Geographical libraries available")
except ImportError:
    geo_available = False
    print("⚠ Geographical libraries not available - using approximate methods")

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully")

✓ Geographical libraries available
Libraries imported successfully


## 2. Load and Examine Anonymous Site Data

In [10]:
# Load the anonymous hospital site data
print("Loading anonymous site data...")
anonymous_file = "../../data/OR_AE2_Project_Adjusted.xlsx"
anonymous_df = pd.read_excel(anonymous_file, engine='openpyxl')

print(f"Dataset loaded: {anonymous_df.shape[0]:,} rows, {anonymous_df.shape[1]} columns")
print("\nColumns:")
print(anonymous_df.columns.tolist())

print("\nFirst 5 rows:")
display(anonymous_df.head())

Loading anonymous site data...
Dataset loaded: 364,346 rows, 19 columns

Columns:
['Site_Code', 'Site_Type', 'Site_X', 'Site_Y', 'Site_Loc_GPs', 'Site_Loc_GP_List', 'Site_Pop_20miles', 'Pat_X', 'Pat_Y', 'Pat_Loc_GPs', 'Pat_Loc_GP_List', 'Drive_Distance_Miles', 'Driving_Time_mins', 'Attendance_Type', 'Age_Group', 'Wait_Time', 'Year', 'Month', 'Number_Of_Attendances']

First 5 rows:
Dataset loaded: 364,346 rows, 19 columns

Columns:
['Site_Code', 'Site_Type', 'Site_X', 'Site_Y', 'Site_Loc_GPs', 'Site_Loc_GP_List', 'Site_Pop_20miles', 'Pat_X', 'Pat_Y', 'Pat_Loc_GPs', 'Pat_Loc_GP_List', 'Drive_Distance_Miles', 'Driving_Time_mins', 'Attendance_Type', 'Age_Group', 'Wait_Time', 'Year', 'Month', 'Number_Of_Attendances']

First 5 rows:


,Site_Code,Site_Type,Site_X,Site_Y,Site_Loc_GPs,Site_Loc_GP_List,Site_Pop_20miles,Pat_X,Pat_Y,Pat_Loc_GPs,Pat_Loc_GP_List,Drive_Distance_Miles,Driving_Time_mins,Attendance_Type,Age_Group,Wait_Time,Year,Month,Number_Of_Attendances
0,2,ED,39785,114688,50,210000,1814482,38971,114101,0,0,00 to 05,00 to 05,New - unplanned,20-39,00-29,1,2,1
1,2,ED,39785,114688,50,210000,1814482,38971,114101,0,0,00 to 05,00 to 05,New - unplanned,20-39,00-29,1,3,2
2,2,ED,39785,114688,50,210000,1814482,38971,114101,0,0,00 to 05,00 to 05,New - unplanned,20-39,00-29,1,4,3
3,2,ED,39785,114688,50,210000,1814482,38971,114101,0,0,00 to 05,00 to 05,New - unplanned,20-39,00-29,1,5,3
4,2,ED,39785,114688,50,210000,1814482,38971,114101,0,0,00 to 05,00 to 05,New - unplanned,20-39,00-29,1,6,2


In [11]:
# Extract unique hospital sites and their characteristics
print("Analyzing anonymous hospital sites...")

# Get unique sites with their coordinates and types
site_columns = ['Site_X', 'Site_Y', 'Site_Type']
if all(col in anonymous_df.columns for col in site_columns):
    unique_sites = anonymous_df[site_columns].drop_duplicates().reset_index(drop=True)
    
    print(f"\nUnique hospital sites found: {len(unique_sites)}")
    print("\nSite types:")
    print(unique_sites['Site_Type'].value_counts())
    
    # Focus on ED and MIU sites (Emergency Department and Minor Injury Unit)
    ed_miu_sites = unique_sites[unique_sites['Site_Type'].isin(['ED', 'MIU'])].copy()
    print(f"\nED/MIU sites: {len(ed_miu_sites)}")
    
    # Add site ID for tracking
    ed_miu_sites['Site_ID'] = range(len(ed_miu_sites))
    
    display(ed_miu_sites)
    
else:
    print("Required columns not found. Available columns:")
    print(anonymous_df.columns.tolist())

Analyzing anonymous hospital sites...

Unique hospital sites found: 13

Site types:
Site_Type
ED           9
MIU/OTHER    4
Name: count, dtype: int64

ED/MIU sites: 9


,Site_X,Site_Y,Site_Type,Site_ID
0,39785,114688,ED,0
1,37920,110782,ED,1
4,34629,117035,ED,2
6,33369,114746,ED,3
7,4258,124892,ED,4
8,27024,111909,ED,5
9,40269,103066,ED,6
10,54562,114442,ED,7
11,57894,104081,ED,8


## 3. Load Real Hospital Data

In [12]:
# Load real hospital data
print("Loading real hospital data...")
hospitals_file = "../../data/hospitals.csv"
hospitals_df = pd.read_csv(hospitals_file)

print(f"Hospital database loaded: {len(hospitals_df)} hospitals")
print("\nColumns:")
print(hospitals_df.columns.tolist())

print("\nFirst 5 hospitals:")
display(hospitals_df[['HospitalCode', 'HospitalName', 'AddressLine1', 'AddressLine2', 'Postcode', 'HealthBoard']].head())

Loading real hospital data...
Hospital database loaded: 245 hospitals

Columns:
['HospitalCode', 'HospitalName', 'AddressLine1', 'AddressLine2', 'AddressLine2QF', 'AddressLine3', 'AddressLine3QF', 'AddressLine4', 'AddressLine4QF', 'Postcode', 'HealthBoard', 'HSCP', 'CouncilArea', 'IntermediateZone', 'DataZone']

First 5 hospitals:


,HospitalCode,HospitalName,AddressLine1,AddressLine2,Postcode,HealthBoard
0,A101H,Arran War Memorial Hospital,Lamlash,Isle of Arran,KA278LF,S08000015
1,A103H,Ayrshire Central Hospital,Kilwinning Road,Irvine,KA128SS,S08000015
2,A110H,Lady Margaret Hospital,College St,Millport,KA280HF,S08000015
3,A111H,University Hospital Crosshouse,Kilmarnock Road,Kilmarnock,KA2 0BE,S08000015
4,A114H,Warrix Avenue Mental Health Community Rehabili...,Warrix Avenue,Irvine,KA120DP,S08000015


In [13]:
# Filter hospitals that might be in Glasgow area
print("Filtering hospitals for Glasgow area...")

# Look for Glasgow-related hospitals
glasgow_keywords = ['glasgow', 'govan', 'gartnavel', 'southern general', 'western infirmary', 
                   'royal infirmary', 'stobhill', 'victoria infirmary', 'beatson']

# Filter by hospital name or address containing Glasgow keywords
glasgow_hospitals = hospitals_df[
    hospitals_df['HospitalName'].str.contains('|'.join(glasgow_keywords), case=False, na=False) |
    hospitals_df['AddressLine1'].str.contains('|'.join(glasgow_keywords), case=False, na=False) |
    hospitals_df['AddressLine2'].str.contains('|'.join(glasgow_keywords), case=False, na=False)
].copy()

print(f"Glasgow area hospitals found: {len(glasgow_hospitals)}")

if len(glasgow_hospitals) > 0:
    print("\nGlasgow hospitals:")
    display(glasgow_hospitals[['HospitalCode', 'HospitalName', 'AddressLine1', 'AddressLine2', 'Postcode']].head(10))
else:
    # If no Glasgow hospitals found by name, try by Health Board
    print("No hospitals found by name, checking by Health Board...")
    print("\nUnique Health Boards:")
    print(hospitals_df['HealthBoard'].value_counts().head(10))
    
    # NHS Greater Glasgow and Clyde should be the Glasgow health board
    glasgow_hb_codes = ['S08000021', 'S08000022']  # Common codes for Glasgow health boards
    glasgow_hospitals = hospitals_df[hospitals_df['HealthBoard'].isin(glasgow_hb_codes)].copy()
    
    if len(glasgow_hospitals) == 0:
        # Show all health boards to identify the correct one
        print("\nAll Health Boards:")
        print(hospitals_df['HealthBoard'].value_counts())

Filtering hospitals for Glasgow area...
Glasgow area hospitals found: 30

Glasgow hospitals:


,HospitalCode,HospitalName,AddressLine1,AddressLine2,Postcode
47,Y146H,Dumfries & Galloway Royal Infirmary,Cargenbridge,Dumfries,DG2 8RX
69,N101H,Aberdeen Royal Infirmary,Foresterhill Road,Aberdeen,AB252ZN
107,G106H,Glasgow Dental Hospital and School,378 Sauchiehall Street,Glasgow,G2 3JZ
108,G107H,Glasgow Royal Infirmary,84 Castle Street,Glasgow,G4 0SF
109,G108H,The Princess Royal Maternity Unit,16 Alexandra Parade,Glasgow,G31 2ER
110,G109H,Lightburn Hospital,966 Carntyne Road,Glasgow,G32 6NB
111,G112H,Parkview Resource Centre,152 Wellshot Road,Glasgow,G32 7AX
113,G207H,Stobhill Hospital,133 Balornock Road,Glasgow,G21 3UW
114,G212H,Shawpark Resource Centre,41 Shawpark Street,Glasgow,G20 9DR
115,G214H,Springpark Resource Centre/Day Hosp,101 Denmark Street,Glasgow,G22 5EU


## 4. Convert Hospital Locations to Project Coordinate System

In [14]:
if geo_available:
    print("Converting hospital locations to project coordinates...")
    
    try:
        # Load the shapefile to get coordinate transformation
        shapefile_path = "../../data/shapefiles/SG_DataZone_Bdry_2011.shp"
        scotland_gdf = gpd.read_file(shapefile_path)
        
        print(f"Shapefile CRS: {scotland_gdf.crs}")
        
        # Get hospitals with DataZones that exist in the shapefile
        hospitals_with_coords = glasgow_hospitals.merge(
            scotland_gdf[['DataZone', 'geometry']], 
            on='DataZone', 
            how='inner'
        )
        
        if len(hospitals_with_coords) > 0:
            # Convert to GeoDataFrame
            hospitals_gdf = gpd.GeoDataFrame(hospitals_with_coords, geometry='geometry')
            
            # Get centroids of DataZones as hospital locations
            hospitals_gdf['centroid'] = hospitals_gdf.geometry.centroid
            
            # Extract X, Y coordinates
            hospitals_gdf['Real_X'] = hospitals_gdf.centroid.x
            hospitals_gdf['Real_Y'] = hospitals_gdf.centroid.y
            
            print(f"Hospitals with coordinates: {len(hospitals_gdf)}")
            
            display(hospitals_gdf[['HospitalCode', 'HospitalName', 'Real_X', 'Real_Y']].head())
            
            coord_conversion_success = True
            
        else:
            print("No hospitals found with matching DataZones in shapefile")
            coord_conversion_success = False
            
    except Exception as e:
        print(f"Error in coordinate conversion: {e}")
        coord_conversion_success = False
        
else:
    print("Geographical libraries not available - using alternative approach")
    coord_conversion_success = False

Converting hospital locations to project coordinates...
Shapefile CRS: EPSG:27700
Hospitals with coordinates: 30
Shapefile CRS: EPSG:27700
Hospitals with coordinates: 30


,HospitalCode,HospitalName,Real_X,Real_Y
0,Y146H,Dumfries & Galloway Royal Infirmary,294320.199066,574164.448031
1,N101H,Aberdeen Royal Infirmary,391911.091735,807049.167790
2,G106H,Glasgow Dental Hospital and School,258143.929583,665846.871054
3,G107H,Glasgow Royal Infirmary,260287.090859,665657.169970
4,G108H,The Princess Royal Maternity Unit,260287.090859,665657.169970


In [19]:
# 5. Hospital Matching Algorithm
print("Starting hospital matching process...")

# Function to calculate Euclidean distance between two points
def calculate_distance(x1, y1, x2, y2):
    """Calculate Euclidean distance between two points"""
    return np.sqrt((x2 - x1)**2 + (y2 - y1)**2)

# Create a mapping for each ED site
site_hospital_mapping = {}
site_details = {}

print("\nMatching each ED site with nearest hospital:")
print("=" * 60)

for idx, site in ed_miu_sites.iterrows():
    site_x, site_y = site['Site_X'], site['Site_Y']
    min_distance = float('inf')
    nearest_hospital = None
    
    # Calculate distance to each hospital
    for idx, hospital in hospitals_with_coords.iterrows():
        # Extract coordinates from geometry
        hospital_x = hospital['geometry'].centroid.x
        hospital_y = hospital['geometry'].centroid.y
        
        distance = calculate_distance(site_x, site_y, 
                                    hospital_x, hospital_y)
        
        if distance < min_distance:
            min_distance = distance
            nearest_hospital = hospital
    
    # Store the mapping
    site_key = (site_x, site_y)
    site_hospital_mapping[site_key] = {
        'hospital_code': nearest_hospital['HospitalCode'],
        'hospital_name': nearest_hospital['HospitalName'],
        'distance_meters': min_distance
    }
    
    print(f"Site {site['Site_ID']} ({site_x:.0f}, {site_y:.0f}) → {nearest_hospital['HospitalName']}")
    print(f"  Code: {nearest_hospital['HospitalCode']}, Distance: {min_distance:.0f}m")
    print()

print(f"Completed matching {len(ed_miu_sites)} ED sites to hospitals")

Starting hospital matching process...

Matching each ED site with nearest hospital:
Site 0 (39785, 114688) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 525268m

Site 1 (37920, 110782) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 529589m

Site 2 (34629, 117035) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 525744m

Site 3 (33369, 114746) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 528357m

Site 4 (4258, 124892) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 534773m

Site 5 (27024, 111909) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 533973m

Site 6 (40269, 103066) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 535234m

Site 7 (54562, 114442) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 518487m

Site 8 (57894, 104081) → Dumfries & Galloway Royal Infirmary
  Code: Y146H, Distance: 526190m

Completed matching 9 ED sites to hospitals


In [18]:
# Debug: Check available data
print("Available variables:")
print(f"hospitals_with_coords columns: {hospitals_with_coords.columns.tolist()}")
print(f"hospitals_with_coords shape: {hospitals_with_coords.shape}")
print("\nFirst few rows:")
print(hospitals_with_coords.head())
print()

Available variables:
hospitals_with_coords columns: ['HospitalCode', 'HospitalName', 'AddressLine1', 'AddressLine2', 'AddressLine2QF', 'AddressLine3', 'AddressLine3QF', 'AddressLine4', 'AddressLine4QF', 'Postcode', 'HealthBoard', 'HSCP', 'CouncilArea', 'IntermediateZone', 'DataZone', 'geometry']
hospitals_with_coords shape: (30, 16)

First few rows:
  HospitalCode                         HospitalName            AddressLine1  \
0        Y146H  Dumfries & Galloway Royal Infirmary            Cargenbridge   
1        N101H             Aberdeen Royal Infirmary       Foresterhill Road   
2        G106H   Glasgow Dental Hospital and School  378 Sauchiehall Street   
3        G107H              Glasgow Royal Infirmary        84 Castle Street   
4        G108H    The Princess Royal Maternity Unit     16 Alexandra Parade   

  AddressLine2 AddressLine2QF AddressLine3 AddressLine3QF AddressLine4  \
0     Dumfries            NaN          NaN              z          NaN   
1     Aberdeen           

In [20]:
# 6. Create Final Dataset with Hospital Information
print("Creating final dataset with hospital assignments...")

# Function to get hospital info for each row
def get_hospital_info(row):
    site_key = (row['Site_X'], row['Site_Y'])
    if site_key in site_hospital_mapping:
        return pd.Series({
            'Hospital_Code': site_hospital_mapping[site_key]['hospital_code'],
            'Hospital_Name': site_hospital_mapping[site_key]['hospital_name'],
            'Distance_to_Hospital_m': site_hospital_mapping[site_key]['distance_meters']
        })
    else:
        return pd.Series({
            'Hospital_Code': 'UNKNOWN',
            'Hospital_Name': 'UNKNOWN',
            'Distance_to_Hospital_m': None
        })

# Apply hospital mapping to the full dataset
print("Adding hospital information to all records...")
hospital_info = anonymous_df.apply(get_hospital_info, axis=1)
final_dataset = pd.concat([anonymous_df, hospital_info], axis=1)

# Display summary
print(f"\nFinal dataset shape: {final_dataset.shape}")
print(f"Records with hospital assignments: {(final_dataset['Hospital_Code'] != 'UNKNOWN').sum()}")

# Show sample of final dataset
print("\nSample of final dataset with hospital information:")
sample_data = final_dataset[['Site_X', 'Site_Y', 'Hospital_Code', 'Hospital_Name', 'Distance_to_Hospital_m']].drop_duplicates().head(10)
print(sample_data)

Creating final dataset with hospital assignments...
Adding hospital information to all records...

Final dataset shape: (364346, 22)
Records with hospital assignments: 341746

Sample of final dataset with hospital information:
        Site_X  Site_Y Hospital_Code                        Hospital_Name  \
0        39785  114688         Y146H  Dumfries & Galloway Royal Infirmary   
1236     37920  110782         Y146H  Dumfries & Galloway Royal Infirmary   
1503     40877  117868       UNKNOWN                              UNKNOWN   
1559     34629  117035         Y146H  Dumfries & Galloway Royal Infirmary   
2515     33369  114746         Y146H  Dumfries & Galloway Royal Infirmary   
3652      4258  124892         Y146H  Dumfries & Galloway Royal Infirmary   
106797   27024  111909         Y146H  Dumfries & Galloway Royal Infirmary   
200574   40269  103066         Y146H  Dumfries & Galloway Royal Infirmary   
248231   54562  114442         Y146H  Dumfries & Galloway Royal Infirmary   
303

In [21]:
# 7. Analysis and Export
print("Analysis of Hospital Assignments:")
print("=" * 50)

# Hospital assignment statistics
assigned_data = final_dataset[final_dataset['Hospital_Code'] != 'UNKNOWN']
print(f"Total records: {len(final_dataset):,}")
print(f"Records with hospital assignments: {len(assigned_data):,}")
print(f"Percentage assigned: {len(assigned_data)/len(final_dataset)*100:.1f}%")

# Distance statistics
distance_stats = assigned_data['Distance_to_Hospital_m'].describe()
print(f"\nDistance Statistics (meters):")
print(f"Mean distance: {distance_stats['mean']:,.0f}m")
print(f"Median distance: {distance_stats['50%']:,.0f}m")
print(f"Min distance: {distance_stats['min']:,.0f}m")
print(f"Max distance: {distance_stats['max']:,.0f}m")

# Hospital assignment counts
print(f"\nHospital Assignment Summary:")
hospital_counts = assigned_data['Hospital_Name'].value_counts()
print(hospital_counts)

# Site type analysis
print(f"\nSite Type Distribution:")
site_type_summary = final_dataset.groupby(['Site_Type', 'Hospital_Name']).size().unstack(fill_value=0)
print(site_type_summary)

# Save the final dataset
output_file = "../../data/FINAL_DATA_with_hospitals.csv"
final_dataset.to_csv(output_file, index=False)
print(f"\n✅ Final dataset saved to: {output_file}")

print(f"\n🎯 Summary:")
print(f"   • Successfully matched {len(ed_miu_sites)} anonymous hospital sites")
print(f"   • Assigned {len(assigned_data):,} patient records to hospitals")
print(f"   • Primary assigned hospital: {hospital_counts.index[0]}")
print(f"   • Average distance to assigned hospital: {distance_stats['mean']:,.0f}m")
print(f"   • Dataset exported with hospital information")

Analysis of Hospital Assignments:
Total records: 364,346
Records with hospital assignments: 341,746
Percentage assigned: 93.8%

Distance Statistics (meters):
Mean distance: 528,281m
Median distance: 528,357m
Min distance: 518,487m
Max distance: 535,234m

Hospital Assignment Summary:
Hospital_Name
Dumfries & Galloway Royal Infirmary    341746
Name: count, dtype: int64

Site Type Distribution:
Hospital_Name  Dumfries & Galloway Royal Infirmary  UNKNOWN
Site_Type                                                  
ED                                          319139        0
MIU/OTHER                                    22607    22600

✅ Final dataset saved to: ../../data/FINAL_DATA_with_hospitals.csv

🎯 Summary:
   • Successfully matched 9 anonymous hospital sites
   • Assigned 341,746 patient records to hospitals
   • Primary assigned hospital: Dumfries & Galloway Royal Infirmary
   • Average distance to assigned hospital: 528,281m
   • Dataset exported with hospital information


In [22]:
# 8. Conclusions and Recommendations

print("\n" + "="*80)
print("ANALYSIS CONCLUSIONS")
print("="*80)

print("\n🔍 COORDINATE SYSTEM ISSUE IDENTIFIED:")
print("   • All ED sites assigned to same hospital with very large distances (500+ km)")
print("   • This suggests the anonymous site coordinates (Site_X, Site_Y) use a different")
print("     coordinate system than the hospital shapefile geometries")
print("   • Site coordinates appear to be in a local/relative system rather than")
print("     British National Grid (EPSG:27700)")

print("\n📊 CURRENT RESULTS:")
print(f"   • Dataset contains {len(final_dataset):,} patient attendance records")
print(f"   • {len(ed_miu_sites)} unique hospital sites identified (9 ED + 4 MIU/OTHER)")
print(f"   • Successfully loaded {len(glasgow_hospitals)} Glasgow-area hospitals for comparison")
print(f"   • Created algorithm that can match sites to hospitals based on proximity")

print("\n💡 RECOMMENDATIONS FOR IMPROVEMENT:")
print("   1. COORDINATE CONVERSION:")
print("      - Determine the actual coordinate system used for Site_X/Site_Y")
print("      - Apply proper coordinate transformation to match hospital system")
print("      - Consider if coordinates are relative to a specific origin point")
print("")
print("   2. ALTERNATIVE MATCHING APPROACHES:")
print("      - Use capacity or volume indicators to match sites to hospitals")
print("      - Consider geographical clustering analysis")
print("      - Match based on service patterns or patient demographics")
print("")
print("   3. VALIDATION:")
print("      - Cross-reference with known hospital locations in the area")
print("      - Validate distance calculations make geographical sense")
print("      - Check if Site_X/Site_Y follow any recognizable pattern")

print("\n✅ DELIVERED:")
print("   • Complete hospital matching framework")
print("   • Association algorithm ready for coordinate correction")
print("   • Final dataset with hospital assignments (pending coordinate fix)")
print("   • Comprehensive analysis and documentation")
print("\n" + "="*80)


ANALYSIS CONCLUSIONS

🔍 COORDINATE SYSTEM ISSUE IDENTIFIED:
   • All ED sites assigned to same hospital with very large distances (500+ km)
   • This suggests the anonymous site coordinates (Site_X, Site_Y) use a different
     coordinate system than the hospital shapefile geometries
   • Site coordinates appear to be in a local/relative system rather than
     British National Grid (EPSG:27700)

📊 CURRENT RESULTS:
   • Dataset contains 364,346 patient attendance records
   • 9 unique hospital sites identified (9 ED + 4 MIU/OTHER)
   • Successfully loaded 30 Glasgow-area hospitals for comparison
   • Created algorithm that can match sites to hospitals based on proximity

💡 RECOMMENDATIONS FOR IMPROVEMENT:
   1. COORDINATE CONVERSION:
      - Determine the actual coordinate system used for Site_X/Site_Y
      - Apply proper coordinate transformation to match hospital system
      - Consider if coordinates are relative to a specific origin point

   2. ALTERNATIVE MATCHING APPROACHES:
 

## 9. Coordinate System Investigation

Let's analyze the coordinate systems used in Site_X/Site_Y and Pat_X/Pat_Y to understand their units and calculation method.

In [1]:
# 9.1 Basic Coordinate Analysis
print("🔍 COORDINATE SYSTEM INVESTIGATION")
print("="*60)

# Check which coordinate columns exist
coord_columns = ['Site_X', 'Site_Y', 'Pat_X', 'Pat_Y']
available_coords = [col for col in coord_columns if col in anonymous_df.columns]
print(f"Available coordinate columns: {available_coords}")

if len(available_coords) > 0:
    print("\n📊 COORDINATE STATISTICS:")
    for col in available_coords:
        print(f"\n{col}:")
        stats = anonymous_df[col].describe()
        print(f"  Range: {stats['min']:,.0f} to {stats['max']:,.0f}")
        print(f"  Mean: {stats['mean']:,.0f}")
        print(f"  Std: {stats['std']:,.0f}")
        print(f"  Non-null values: {anonymous_df[col].notna().sum():,}")
        
    # Show sample of coordinate data
    print(f"\n📋 SAMPLE COORDINATE DATA:")
    sample_coords = anonymous_df[available_coords].head(10)
    display(sample_coords)
else:
    print("❌ No coordinate columns found!")
    print("Available columns:", anonymous_df.columns.tolist())

🔍 COORDINATE SYSTEM INVESTIGATION


NameError: name 'anonymous_df' is not defined